In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import humanize
import seaborn as sns

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 200)


### Pre-processing 

In [37]:
# load EAVS/VV pre-merged file
eavs = pd.read_pickle('data/merged_eavs_and_vv_data.pkl.zip')

In [38]:
# coerce non-equipment columns
def coerce_column(col):
    col = pd.to_numeric(col, errors='coerce')
    col.loc[col < 0] = np.nan  # set negative values to NaN
    return col

numeric_columns = ['A1a', 'D1a', 'D2a', 'D3a', 'D4a', 'F1a', 'F1b', 'F1d', 'F1e', 'F1f', 'F1g']
for col in numeric_columns:
    eavs[col] = coerce_column(eavs[col])

# Coerce equipment count columns
for equipment in ['3', '4', '5', '6']:
    for i in range(1, 4):
        col = f'F{equipment}c_{i}'
        if col in eavs.columns:
            eavs[col] = coerce_column(eavs[col])



In [39]:
cols = {'A1a': 'registered voters',
        'F1a': 'ballots cast',
        'D1a': 'precincts',
        'D2a': 'polling places 2024 general election',
        'D3a': 'polling places 2024 election day',
        'D4a': 'polling places 2024 early voting'
        }

for col, label in cols.items():
    print(f'{humanize.intword(eavs[col].sum())} {label}')


235.1 million registered voters
158.2 million ballots cast
177.7 thousand precincts
95.8 thousand polling places 2024 general election
94.9 thousand polling places 2024 election day
30.5 thousand polling places 2024 early voting


### Tally equipment

In [40]:
# indicate whether scanners are hand-fed or batch-fed

scanners = {
    'AccuVote-OS (Premier)': 'hand-fed',
    'BallotNow (Hart)': 'batch-fed',
    'Chatsworth ACP (MicroVotes)': 'hand-fed',
    'ClearCast (Clear Ballot)': 'hand-fed',
    'ClearCount (Clear Ballot)': 'batch-fed',
    'DS200 (ES&S)': 'hand-fed',
    'DS300 (ES&S)': 'hand-fed',
    'DS450 (ES&S)': 'batch-fed',
    'DS850 (ES&S)': 'batch-fed',
    'DS950 (ES&S)': 'batch-fed',
    'ExpressVote Tabulator (ES&S)': 'hand-fed',
    'ExpressVoteXL (ES&S)': 'hand-fed',
    'IBML (Los Angeles County)': 'batch-fed',
    'ImageCast Central/ICC (Dominion)': 'batch-fed',
    'ImageCast Evolution/ICE (Dominion)': 'hand-fed',
    'ImageCast Precinct/ICP (Dominion)': 'hand-fed',
    'ImageCast Precint2/ICP2': 'hand-fed',
    'M100 (ES&S)': 'hand-fed',
    'M650 (ES&S)': 'batch-fed',
    'OpenElect Freedom Vote Scan (Unisyn)': 'hand-fed',
    'OpenElect Voting Central Scan/OVCS (Unisyn)': 'batch-fed',
    'OpenElect Voting Optical Scan/OVO (Unisyn)': 'hand-fed',
    'Verity Central (Hart)': 'batch-fed',
    'Verity Scan (Hart)': 'hand-fed',
    'eScan (Hart)': 'hand-fed'
}


EAVS question format on equipment

F[#] Codes:

F3 → DRE without VVPAT

F4 → DRE with VVPAT

F5 → Ballot Marking Device (BMD)

F6 → Scanner
```
F[#] — Device Category
├── a           → Is this device type used? (Yes/No)
├── b_1         → Model of first device (if any)
│   └── c_1     → Quantity of that model
├── b_2         → Model of second device (if any)
│   └── c_2     → Quantity of that model
└── b_3         → Model of third device (if any)
    └── c_3     → Quantity of that model
├── d_1         → This equipment supported
                  In-precinct Election Day
                  regular ballot marking
                  and/or counting (used by all voters)
├── d_2         → This equipment supported
                  In-precinct accessible voting
                  primarily for voters with disabilities
├── d_3         → This equipment supported
                  Provisional ballot marking and/or
                  counting
├── d_4         → This equipment supported
                  In-person early voting
                  ballot marking and/or counting
├── d_5         → This equipment supported
                  Mail ballot counting only
                  (F5 and F6: BMDs and Scanners only)
```

- Dominion ICE? it's a hybrid scanner/BMD. can we tell if most jurisdictions just use it as a scanner? and assume that?
    - some jurisdictions treat it as a BMD and some as a scanner
- Same with the hart A/T.
    - eScan is always treated as a scanner
- ExpressVote XL: let's treat it as just a BMD.
    - Sometimes treated as a BMD and sometimes as a scanner
- Let's exclude hybrids from analysis.

In [41]:
models = [f'F{equipment}b_{number}{suffix}' for equipment in range(3,7) for number in range(1,4) for suffix in ['', 'other']]
models = [f'F{equipment}b_{number}{suffix}' for equipment in range(3,7) for number in range(1,4) for suffix in ['']]

all_models = pd.DataFrame(eavs[models].melt()['value'].value_counts())

all_models.head()

,count
value,
Valid skip,40927
Does not apply,7676
Data not available,2630
DS200 (ES&S),2170
ExpressVote (ES&S),1651


In [42]:
# models = [f'F{equipment}b_{number}{suffix}' for equipment in range(3,7) for number in range(1,4) for suffix in ['', 'other']]
models = [f'F{equipment}b_{number}{suffix}' for equipment in range(3,7) for number in range(1,4) for suffix in ['other']]

all_models = pd.DataFrame(eavs[models].melt()['value'].value_counts())
all_models

,count
value,
VALID SKIP,41058
DOES NOT APPLY,1511
DNA (DATA NOT AVAILABLE),705
"ACCUVOTE/A,B,C,D-1.96.13, DOMINION",177
MICROVOTE EMS 4.41 VOTING SYSTEM,56
...,...
5.12.6.60,1
5.12.6.59,1
5.12.6.58,1


In [43]:
# count devices for each type of equipment

def count_devices(row, prefix):
    total = 0
    has_data = False

    for i in range(1, 4):
        model = row.get(f'{prefix}b_{i}')
        count = row.get(f'{prefix}c_{i}')
        if pd.isna(model) or pd.isna(count):
            continue
        has_data = True
        total += count

    if not has_data:
        return np.nan
    return total

equipment_codes = {'dre_wo_vvpat': 'F3',
                   'dre_w_vvpat': 'F4',
                   'bmd': 'F5',
                   'scanner': 'F6'}

for equipment, prefix in equipment_codes.items():
    eavs[f'{equipment}_total'] = eavs.apply(lambda row: count_devices(row, prefix), axis=1)

In [44]:

# count hand-fed and batch-fed totals for a row
def count_scanners(row):
    hand_total = 0
    batch_total = 0
    has_hand = False
    has_batch = False
    has_data = False

    for i in range(1, 4):
        model = row.get(f'F6b_{i}')
        count = row.get(f'F6c_{i}')
        if pd.isna(model) or pd.isna(count):
            continue
        has_data = True

        feed_type = scanners.get(model, 'unknown')
        if feed_type == 'unknown':
            return pd.Series({'hand_fed_total': np.nan, 'batch_fed_total': np.nan})
        elif feed_type == 'hand-fed':
            hand_total += count
            has_hand = True
        elif feed_type == 'batch-fed':
            batch_total += count
            has_batch = True

    if not has_data:
        return pd.Series({'hand_fed_total': np.nan, 'batch_fed_total': np.nan})

    return pd.Series({
        'hand_fed_total': hand_total if has_hand else np.nan,
        'batch_fed_total': batch_total if has_batch else np.nan
    })

# Apply function row-wise
eavs[['hand_fed_total', 'batch_fed_total']] = eavs.apply(count_scanners, axis=1)


In [45]:
def incomplete_equipment_data(row):
    incomplete = False
    # identifies rows where equipment is marked as 'Yes' but no quantity is provided

    for equipment in ['3', '4', '5', '6']:
        if row[f'F{equipment}a'] == 'Yes':
            # only check the first model for each equipment type
            if pd.isna(row[f'F{equipment}c_1']): # to require model, also use pd.isna(row[f'F{equipment}b_1'])
                incomplete = True

    return incomplete

eavs['incomplete_equipment'] = eavs.apply(incomplete_equipment_data, axis=1)

equipment_counts = [key + '_total' for key in equipment_codes.keys()][:-1] + ['hand_fed_total', 'batch_fed_total']
eavs['total_equipment_counts'] = eavs[equipment_counts].sum(axis=1)

eavs['equipment_per_voter'] = eavs['total_equipment_counts'] / eavs['A1a']

## Apply filters to get overall equipment rates

In [88]:
hybrid_models = ['ExpressVoteXL (ES&S)', 'ES&S EXPRESSVOTE XL', 'ImageCast Evolution/ICE (Dominion)', 'eScan (Hart)']

drop_masks = {'complete equipment': eavs['incomplete_equipment'],
            'zero equipment reports': eavs['total_equipment_counts'] == 0,
            # 'hybrid models': eavs.apply(lambda r: r.isin(hybrid_models).any(), axis=1),
            # 'DRE for all': eavs['dre_for_all']
            }

removed_mask = pd.Series(False, index=eavs.index)

for i, (name, mask) in enumerate(drop_masks.items(), start=1):
    delta = mask & ~removed_mask
    print(f"Mask {i} ({name}) removes an additional: {delta.sum()} jurisdictions, representing {humanize.intword(eavs[delta]['A1a'].sum())} registered voters")
    removed_mask |= mask
    print(sum(removed_mask))

voters_remaining = eavs[~removed_mask]['A1a'].sum()
voters_total = eavs['A1a'].sum()
print(f"\nAfter all masks, there remain {sum(~removed_mask)} of {len(removed_mask)} jurisdictions ({len(removed_mask) - sum(~removed_mask)} were removed), representing {humanize.intword(voters_remaining)} of {humanize.intword(voters_total)} ({voters_remaining / voters_total:.1%}) registered voters")

eavs['use_for_analysis'] = ~removed_mask
eq = eavs[eavs['use_for_analysis']]

Mask 1 (complete equipment) removes an additional: 2443 jurisdictions, representing 35.3 million registered voters
2443
Mask 2 (zero equipment reports) removes an additional: 401 jurisdictions, representing 4.9 million registered voters
2844

After all masks, there remain 3617 of 6461 jurisdictions (2844 were removed), representing 194.8 million of 235.1 million (82.9%) registered voters


## Show number of voters per piece of equipment

In [87]:
counts = eq.groupby(['Election Day Marking Method'])[equipment_counts].sum()
pop = eq.groupby(['Election Day Marking Method'])['A1a'].sum()
per_voter_rates = counts.div(pop, axis=0)
per_voter_rates.to_csv('data/equipment_per_voter_rates_by_marking_method.csv')
(1/per_voter_rates).map(humanize.intword)

,dre_wo_vvpat_total,dre_w_vvpat_total,bmd_total,hand_fed_total,batch_fed_total
Election Day Marking Method,,,,,
Ballot Marking Devices for all voters,83.9 thousand,27.7 thousand,374,2.1 thousand,114.4 thousand
DREs with VVPAT for all voters,+Inf,369,49.2 thousand,124.4 thousand,146.6 thousand
DREs without VVPAT for all voters,318,+Inf,+Inf,612.3 thousand,+Inf
Hand marked paper ballots and BMDs,764.8 thousand,517.3 thousand,2.0 thousand,1.9 thousand,83.6 thousand
Hand marked paper ballots and DREs with VVPAT,+Inf,927,1.8 thousand,+Inf,+Inf
Hand marked paper ballots and DREs without VVPAT,1.7 thousand,+Inf,+Inf,1.7 thousand,6.0 thousand
"Hand marked paper ballots, BMDs and DREs with VVPAT",+Inf,4.2 thousand,2.2 thousand,2.1 thousand,55.6 thousand
"Hand marked paper ballots, BMDs and DREs without VVPAT",4.4 thousand,+Inf,598,4.1 thousand,187.2 thousand
Hand marked paper ballots; Direct recording assistive interface without VVPAT for accessibility,+Inf,+Inf,+Inf,1.2 thousand,610.6 thousand


## Apply filters for EAVS cost analysis

In [89]:
hybrid_models = ['ExpressVoteXL (ES&S)', 'ES&S EXPRESSVOTE XL', 'ImageCast Evolution/ICE (Dominion)', 'eScan (Hart)']

drop_masks = {'complete equipment': eavs['incomplete_equipment'],
            'zero equipment reports': eavs['total_equipment_counts'] == 0,
            'hybrid models': eavs.apply(lambda r: r.isin(hybrid_models).any(), axis=1),
            'DRE for all': eavs['dre_for_all']
            }

removed_mask = pd.Series(False, index=eavs.index)

for i, (name, mask) in enumerate(drop_masks.items(), start=1):
    delta = mask & ~removed_mask
    print(f"Mask {i} ({name}) removes an additional: {delta.sum()} jurisdictions, representing {humanize.intword(eavs[delta]['A1a'].sum())} registered voters")
    removed_mask |= mask
    print(sum(removed_mask))

voters_remaining = eavs[~removed_mask]['A1a'].sum()
voters_total = eavs['A1a'].sum()
print(f"\nAfter all masks, there remain {sum(~removed_mask)} of {len(removed_mask)} jurisdictions ({len(removed_mask) - sum(~removed_mask)} were removed), representing {humanize.intword(voters_remaining)} of {humanize.intword(voters_total)} ({voters_remaining / voters_total:.1%}) registered voters")

eavs['use_for_analysis'] = ~removed_mask
eq = eavs[eavs['use_for_analysis']]

Mask 1 (complete equipment) removes an additional: 2443 jurisdictions, representing 35.3 million registered voters
2443
Mask 2 (zero equipment reports) removes an additional: 401 jurisdictions, representing 4.9 million registered voters
2844
Mask 3 (hybrid models) removes an additional: 199 jurisdictions, representing 17.0 million registered voters
3043
Mask 4 (DRE for all) removes an additional: 189 jurisdictions, representing 11.3 million registered voters
3232

After all masks, there remain 3229 of 6461 jurisdictions (3232 were removed), representing 166.6 million of 235.1 million (70.9%) registered voters


In [90]:
grouped = eq.groupby(['bmd_for_all'])[equipment_counts + ['scanner_total'] + ['A1a']].sum()
per_voter_rates = grouped[equipment_counts].div(grouped['A1a'], axis=0)
per_voter_rates


,dre_wo_vvpat_total,dre_w_vvpat_total,bmd_total,hand_fed_total,batch_fed_total
bmd_for_all,,,,,
False,0.000003,0.000003,0.000499,0.000509,0.000012
True,0.000014,0.000013,0.002817,0.000460,0.000009


Checks:
- Could I use EAVS instead of VV to determine BMD for all vs no? no.
- Is EAVS equipment type usable? as in mail vs others... *wouldn't change things. i think we need a caveat that says we ignore mail-in policies and also what the equipment was used for.*

## extrapolate findings to the rows that were not used for analysis

In [91]:
eavs = eavs.copy()

# Apply idealized estimates only to rows not used for analysis
mask = ~eavs['use_for_analysis']

# Estimate based on whether the jurisdiction uses BMDs for all voters
for bmd_flag in [True, False]:
    rate_row = per_voter_rates.loc[bmd_flag]
    submask = mask & (eavs['bmd_or_dre_for_all'] == bmd_flag)
    for equipment in equipment_counts:
        eavs.loc[submask, f'{equipment}_ideal'] = eavs.loc[submask, 'A1a'] * rate_row[equipment]

# # For rows used in analysis, copy actuals into the _ideal columns
for equipment in equipment_counts:
    eavs.loc[eavs['use_for_analysis'], f'{equipment}_ideal'] = eavs.loc[eavs['use_for_analysis'], equipment]

eavs['bmd_total_ideal'] = (
    eavs.get('bmd_total_ideal', 0).fillna(0) +
    eavs.get('dre_wo_vvpat_total_ideal', 0).fillna(0) +
    eavs.get('dre_w_vvpat_total_ideal', 0).fillna(0)
)


In [92]:
cbo = pd.read_excel('data/cbo_projections.xlsx',
                    sheet_name='1. Econ Vars_Annual Rates',
                    header=7, index_col=0).T

inflation_rates = cbo['Growth of the CPI-U'] / 100

def inflation_multiplier(inflation_series, start_year, end_year):
    """Calculate compounded inflation multiplier from start_year to end_year (inclusive)."""
    relevant_years = inflation_series.loc[start_year:end_year]
    multiplier = (1 + relevant_years).prod()
    return multiplier


In [93]:
# vvsg_multiplier = 1.2  # multiplier from old 20-50% range

start_year = 2026

replacement_year = 2028

inflation = inflation_multiplier(inflation_rates, start_year, replacement_year)

# cost_multiplier = vvsg_multiplier * inflation

# starting point: median prices on caulfield estimates; see historical_prices.ipynb

base_2021 = {
    "BMD": 3665,
    "Hand-fed scanner": 5611,
    "Batch-fed scanner": 80157,
}

# acet ranges for 2026: scanners: 20-30% more expensive than in 2021
#                       BMDs: 35-50% more expensive due to display requirements
ranges_2026 = {
    "BMD": (1.35, 1.5),
    "Hand-fed scanner": (1.2, 1.3),
    "Batch-fed scanner": (1.2, 1.3),
}

rows = []
for equip in ["BMD", "Hand-fed scanner", "Batch-fed scanner"]:
    base = base_2021[equip]
    low_mult, high_mult = ranges_2026[equip]
    low  = base * low_mult  * inflation
    high = base * high_mult * inflation
    mean = base * np.mean([low_mult, high_mult]) * inflation

    rows.append({
        "equipment": equip.replace("_", " "),
        "2021 mean": base,
        f"{replacement_year} low": low,
        f"{replacement_year} high": high,
        f"{replacement_year} mean": mean,
    })

df = pd.DataFrame(rows).set_index("equipment")

# Pretty display columns (currency + range)
def fmt(x): return f"${x:,.0f}"

df_display = pd.DataFrame({
    "2021 mean": df["2021 mean"].map(fmt),
    f"{replacement_year} estimate": (df[f"{replacement_year} low"].map(fmt) + " – " + df[f"{replacement_year} high"].map(fmt)),
    f"{replacement_year} mean": df[f"{replacement_year} mean"].map(fmt),
})

df_display


,2021 mean,2028 estimate,2028 mean
equipment,,,
BMD,"$3,665","$5,303 – $5,892","$5,598"
Hand-fed scanner,"$5,611","$7,217 – $7,818","$7,517"
Batch-fed scanner,"$80,157","$103,093 – $111,684","$107,388"


In [94]:

# Build table rows
rows = []

for label, group in eavs.groupby('bmd_or_dre_for_all'):
    jurisdictions = len(group)

    n_registered_voters = group['A1a'].sum()

    bmd_count = group['bmd_total_ideal'].sum()
    hand_fed_count = group['hand_fed_total_ideal'].sum()
    batch_fed_count = group['batch_fed_total_ideal'].sum()

    bmd_total = bmd_count * df.loc['BMD', f'{replacement_year} mean']
    hand_fed_total = hand_fed_count * df.loc['Hand-fed scanner', f'{replacement_year} mean']
    batch_fed_total = batch_fed_count * df.loc['Batch-fed scanner', f'{replacement_year} mean']
    total_cost = bmd_total + hand_fed_total + batch_fed_total

    rows.append({
        'bmd_or_dre_for_all': label,
        '# Jurisdictions': jurisdictions,
        '# Registered Voters': n_registered_voters,
        '# BMDs': int(bmd_count),
        'BMD Total Cost': bmd_total,
        '# Hand-Fed Scanners': int(hand_fed_count),
        'Hand-Fed Scanner Total Cost': hand_fed_total,
        '# Batch-Fed Scanners': int(batch_fed_count),
        'Batch-Fed Scanner Total Cost': batch_fed_total,
        'Total Cost': total_cost,
    })

# Create DataFrame
cost_table = pd.DataFrame(rows)

# Add total row
totals = {
    'bmd_or_dre_for_all': 'Total',
    '# Registered Voters': cost_table['# Registered Voters'].sum(),
    '# Jurisdictions': cost_table['# Jurisdictions'].sum(),
    '# BMDs': cost_table['# BMDs'].sum(),
    'BMD Total Cost': cost_table['BMD Total Cost'].sum(),
    '# Hand-Fed Scanners': cost_table['# Hand-Fed Scanners'].sum(),
    'Hand-Fed Scanner Total Cost': cost_table['Hand-Fed Scanner Total Cost'].sum(),
    '# Batch-Fed Scanners': cost_table['# Batch-Fed Scanners'].sum(),
    'Batch-Fed Scanner Total Cost': cost_table['Batch-Fed Scanner Total Cost'].sum(),
    'Total Cost': cost_table['Total Cost'].sum(),
}
cost_table = pd.concat([cost_table, pd.DataFrame([totals])], ignore_index=True)

# Custom formatter: shows M or B with 1 decimal when needed
def number_formatter(x, pos=None):
    units = [(1e12, 'T'), (1e9, 'B'), (1e6, 'M'), (1e3, 'K')]

    for factor, suffix in units:
        if x >= factor:
            return f'{x / factor:.3g} {suffix}'.rstrip('0').rstrip('.')

    return f'{x:.0f}'

# Format cost columns
for col in ['BMD Total Cost', 'Hand-Fed Scanner Total Cost', 'Batch-Fed Scanner Total Cost', 'Total Cost']:
    # cost_table[col] = cost_table[col].map('${:,.0f}'.format)
    # cost_table[col] = cost_table[col].apply(lamda x: "humanize.intword)
    cost_table[col] = cost_table[col].apply(lambda x: f'${number_formatter(x)}')

# Format number columns
for col in ['# Jurisdictions', '# Registered Voters', '# BMDs', '# Hand-Fed Scanners', '# Batch-Fed Scanners']:
    cost_table[col] = cost_table[col].apply(number_formatter)

cost_table

cost_table = cost_table.set_index('bmd_or_dre_for_all')
cost_table.index = cost_table.index.map({
    True: 'BMD/DRE-for-all jurisdictions',
    False: 'HMPB+BMD jurisdictions',
    'Total': 'Total'
})
cost_table.index.name = None  # Remove index title

# Reorder rows
cost_table = cost_table.reindex([
    'BMD/DRE-for-all jurisdictions',
    'HMPB+BMD jurisdictions',
    'Total'
])

cost_table.index.name = None  # Remove index title

cost_table

,# Jurisdictions,# Registered Voters,# BMDs,BMD Total Cost,# Hand-Fed Scanners,Hand-Fed Scanner Total Cost,# Batch-Fed Scanners,Batch-Fed Scanner Total Cost,Total Cost
BMD/DRE-for-all jurisdictions,839,70.9 M,202 K,$1.13 B,32.6 K,$245 M,620,$66.7 M,$1.44 B
HMPB+BMD jurisdictions,5.62 K,164 M,83 K,$465 M,83.5 K,$628 M,2.04 K,$219 M,$1.31 B
Total,6.46 K,235 M,285 K,$1.59 B,116 K,$873 M,2.66 K,$286 M,$2.75 B


In [95]:
cost_table[['# Jurisdictions', '# Registered Voters', 'BMD Total Cost', 'Hand-Fed Scanner Total Cost', 'Batch-Fed Scanner Total Cost', 'Total Cost']]

,# Jurisdictions,# Registered Voters,BMD Total Cost,Hand-Fed Scanner Total Cost,Batch-Fed Scanner Total Cost,Total Cost
BMD/DRE-for-all jurisdictions,839,70.9 M,$1.13 B,$245 M,$66.7 M,$1.44 B
HMPB+BMD jurisdictions,5.62 K,164 M,$465 M,$628 M,$219 M,$1.31 B
Total,6.46 K,235 M,$1.59 B,$873 M,$286 M,$2.75 B


## Analysis (flawed) of central vs precinct count

can't really use this.

In [11]:
eavs['batch_fed_total'].sum()

np.float64(2126.0)

In [12]:
has_scanner = eavs['F6a'] == 'Yes'

eavs[has_scanner]['F6b_1'].value_counts() + eavs[has_scanner]['F6b_2'].value_counts() + eavs[has_scanner]['F6b_3'].value_counts()

AccuVote-OS (Premier)                             NaN
BallotNow (Hart)                                  NaN
Chatsworth ACP (MicroVotes)                       NaN
ClearCast (Clear Ballot)                          NaN
ClearCount (Clear Ballot)                        49.0
DS200 (ES&S)                                   2170.0
DS300 (ES&S)                                    107.0
DS450 (ES&S)                                    413.0
DS850 (ES&S)                                    176.0
DS950 (ES&S)                                     39.0
Data not available                              378.0
Does not apply                                    NaN
ExpressVote Tabulator (ES&S)                     13.0
ExpressVoteXL (ES&S)                              NaN
IBML (Los Angeles County)                         NaN
ImageCast Central/ICC (Dominion)                407.0
ImageCast Evolution/ICE (Dominion)                NaN
ImageCast Precinct/ICP (Dominion)                 NaN
ImageCast Precint2/ICP2     

do batch-fed scanners as a function of number of mail-in ballots?

In [13]:
pd.to_numeric(eavs['C8a'], errors='coerce').sum()

np.float64(47629437.0)

In [ ]:
eavs['ballots_counted_centrally'] = (eavs.loc[eavs['F12a'] == 'Central location', 'F1b'] + # ballots cast in person on election day
    eavs.loc[eavs['F12c'] == 'Central location', 'F1e'] + # provisional ballots
    eavs.loc[eavs['F12d'] == 'Central location', 'F1f'] + # ballots cast early in person
    eavs.loc[eavs['F12e'] == 'Central location', 'F1d'] + # mail ballots, jurisdictions that do not send mail ballots to all voters
    eavs.loc[eavs['F12e'] == 'Central location', 'F1g']) #  mail ballots, jurisdictions that send mail ballots to all voters

humanize.intword(eavs['ballots_counted_centrally'].sum())


# F1b/e/f/g (ballots counted) * F12a/c/d/e

'60.8 million'

In [ ]:
# % of total ballots (F1a) counted centrally. this is way off because jurisdictions are incorrectly reporting a central scan
(eavs.groupby('State_Full')['ballots_counted_centrally'].sum().sort_values(ascending=False) / 
  eavs.groupby('State_Full')['F1a'].sum().sort_values(ascending=False)).sort_values(ascending=False)*100

State_Full
NORTHERN MARIANA ISLANDS    100.000000
HAWAII                      100.000000
GUAM                         99.838193
LOUISIANA                    99.795705
WEST VIRGINIA                99.779773
OKLAHOMA                     99.732087
DELAWARE                     99.673191
ILLINOIS                     99.558609
WYOMING                      99.512767
NEW YORK                     99.393537
MARYLAND                     99.381705
NEW MEXICO                   99.377642
NEVADA                       99.248939
COLORADO                     99.005355
NEBRASKA                     97.631651
CALIFORNIA                   83.172522
SOUTH DAKOTA                 82.570530
UTAH                         74.920785
TEXAS                        70.363092
KENTUCKY                     49.051815
MISSOURI                     45.564064
OHIO                         44.184359
ARIZONA                      35.581653
ARKANSAS                     22.799520
FLORIDA                      18.397282
INDIANA       